In [ ]:
"""
================================================================================
AHA IT Survey 2023 - Interactive Map Visualization
================================================================================

This script creates an interactive map visualization of hospital data from the 
AHA IT Survey 2023, with toggleable choropleth layers.

DATA REQUIREMENTS:
------------------
Users must download and place the following data files:

1. AHA 2023 Master Data (CSV) - REQUIRED
   - File: AHA2023_master.csv
   - Source: American Hospital Association (AHA) Annual Survey 
   - Required columns: See section below

2. Geographic Choropleth Data - OPTIONAL (you can add multiple layers)
   The script supports loading multiple choropleth layers which can be toggled
   in the interactive map. Examples:
   

   
   Example A: HPSA Scores
   - File: Your HPSA SHP file 
   - Fields: GEOID, HPSA_SCORE (or similar)



HOW TO ADD CHOROPLETH LAYERS:
-----------------------------
Edit the GEO_LAYERS configuration below to add your choropleth layers:

GEO_LAYERS = [
    # Add more layers like this:
    # {
    #     'name': 'Elder Poverty',
    #     'file': './data/geodata/elder_poverty.geojson',
    #     'id_field': 'GEOID',
    #     'value_field': 'elder_poverty_rate',
    #     'show_by_default': False
    # }
]



USAGE:
------
1. Download AHA Annual Survey and IT Supplement 
2. Create master file 
2.1 Convert annual survey columns into lower case and rename by appending '_as'. 
2.2 Convert IT survey columns into lower case and rename by appending '_it'. 
2.3 Merge the two files by 'id_as' and 'id_it'. 
2.4 Create a new column 'answered' to indicate if the hospital responded to the IT survey. 
2.5 Create a new column 'id_as' to indicate the hospital ID. 
3. Optionally add your choropleth data files
4. Update CONFIG section with your file paths
5. Run: python AHA_IT_2023_Interactive_Map.py
6. Output: HTML file will be saved and opened in browser
================================================================================
"""

In [ ]:
# Cell 1: Load Data
import pandas as pd
import numpy as np
import geopandas as gpd
import folium




# Cell 2: Configuration and Functions
# Configuration
MAP_CENTER = [37.7749, -122.4194]
MAP_ZOOM = 6
MAP_TILES = 'CartoDB positron'
OUTPUT_FILE = './US_aha_hpsa_map.html'


def prepare_aha_data(AHA2023):
    """Clean AHA data"""
    print("Preparing AHA data...")
    df = AHA2023.copy()
    
    # Create 'answered' column
    df['answered'] = df['id_it'].notnull() if 'id_it' in df.columns else True
    
    # Convert ID to string
    if 'id_as' in df.columns:
        df['id_as'] = df['id_as'].astype(str).str.strip()
    
    # Drop missing coordinates
    initial_count = len(df)
    df = df.dropna(subset=['lat_as', 'long_as'])
    
    # Convert coordinates to numeric
    df['latitude_address'] = pd.to_numeric(df['lat_as'], errors='coerce')
    df['longitude_address'] = pd.to_numeric(df['long_as'], errors='coerce')
    
    print(f"✓ {len(df)} hospitals ({initial_count - len(df)} dropped)")
    return df

# example chloropleth layer - collected from the HRSA website

def prepare_hpsa_layers(hpsa_primary, hpsa_dental, hpsa_mental):
    """Prepare HPSA layers"""
    print("\nPreparing HPSA choropleth layers...")
    layers = []
    
    for name, hpsa_data, field_name in [
        ('HPSA Primary Care', hpsa_primary, 'HpsScore'),
        ('HPSA Dental', hpsa_dental, 'HpsScore'),
        ('HPSA Mental Health', hpsa_mental, 'HpsScore')
    ]:
        try:
            hpsa_data = hpsa_data.to_crs("EPSG:4326").copy()
            
            # Convert Timestamp columns to strings
            for col in hpsa_data.columns:
                if pd.api.types.is_datetime64_any_dtype(hpsa_data[col]):
                    hpsa_data[col] = hpsa_data[col].astype(str)
            
            # Find the actual ID column
            id_field = 'FID' if 'FID' in hpsa_data.columns else 'OBJECTID' if 'OBJECTID' in hpsa_data.columns else 'id'
            
            # Create a unique ID if none exists
            if id_field not in hpsa_data.columns:
                hpsa_data[id_field] = range(len(hpsa_data))
            
            hpsa_data['score'] = pd.to_numeric(hpsa_data[field_name], errors='coerce')
            hpsa_filtered = hpsa_data.dropna(subset=['score'])
            
            if len(hpsa_filtered) > 0:
                geojson_str = hpsa_filtered.to_json()
                layers.append({
                    'name': name,
                    'geojson': geojson_str,
                    'data': hpsa_filtered,
                    'id_field': id_field,
                    'value_field': 'score'
                })
                print(f"✓ {name}: {len(hpsa_filtered)} areas (using {id_field} as ID)")
        except Exception as e:
            print(f"⚠ {name}: {e}")
            import traceback
            traceback.print_exc()
    
    return layers

In [ ]:
AHA2023 = pd.read_csv('AHA2023_master.csv', low_memory=False)
hpsa_primary = gpd.read_file('../../data/hpsa_primary/HPSA_PCP.shp')
hpsa_dental = gpd.read_file('../../data/hpsa_dental/HPSA_DH.shp')
hpsa_mental = gpd.read_file('../../data/hpsa_mental/HPSA_MH.shp')


In [ ]:
piemr_mapping = {
        5: 'Epic',
        2: 'Cerner',
        9: 'Meditech',
        17: 'CPSI/Evident',
        25: 'Other',
        29: 'Altera'
    }
def map_piemr(value):
        if pd.isna(value):
            return None  # Keep as missing
        return piemr_mapping.get(value, 'Other')  # Map to 'Other' if not found

    # Apply the mapping to create the 'piemr_mapped' column
AHA2023['piemr_it_mapped'] = AHA2023['piemr_it'].apply(map_piemr)
AHA2023['piemr_it_mapped'].unique()

aipred_mapping = {1:'ML',
                  2:'Other non-ML predictive model',
                  3: 'Neither/Do not know',
                  4: 'Neither/Do not know'}
AHA2023['aipred_it_mapped'] = AHA2023['aipred_it'].map(aipred_mapping)

In [ ]:
def create_hospital_marker_with_controls(row):
    """Create marker HTML with all data attributes for JavaScript controls"""
    def get_data_attr(col_name, binary=False):
        if col_name not in row.index or pd.isna(row[col_name]):
            return ''
        if binary:
            return '1' if row[col_name] == 1 else '0'
        return str(row[col_name])
    
    icon_html = f"""
        <div class="hospital-marker circle-marker" 
             style="width: 10px; height: 10px; background-color: #666666;
                    border: 1px solid black; border-radius: 50%;"
             data-ai="{get_data_attr('aipred_it')}"
             data-response="{str(row.get('answered', False)).lower()}"
             data-aitraj="{get_data_attr('aitraj_it', binary=True)}"
             data-airfol="{get_data_attr('airfol_it', binary=True)}"
             data-aimhea="{get_data_attr('aimhea_it', binary=True)}"
             data-airect="{get_data_attr('airect_it', binary=True)}"
             data-aibill="{get_data_attr('aibill_it', binary=True)}"
             data-aische="{get_data_attr('aische_it', binary=True)}"
             data-aipoth="{get_data_attr('aipoth_it', binary=True)}"
             data-aicloth="{get_data_attr('aicloth_it', binary=True)}"
             data-ainota="{get_data_attr('ainota_it', binary=True)}"
             data-equery="{get_data_attr('equery_it', binary=True)}"
             data-eadhg="{get_data_attr('eadhg_it', binary=True)}"
             data-eidgap="{get_data_attr('eidgap_it', binary=True)}"
             data-esimp="{get_data_attr('esimp_it', binary=True)}"
             data-emsaf="{get_data_attr('emsaf_it', binary=True)}"
             data-eralg="{get_data_attr('eralg_it', binary=True)}"
             data-enota="{get_data_attr('enota_it', binary=True)}"
             data-piemr="{get_data_attr('piemr_it_mapped')}"
             data-mldev="{get_data_attr('mldev_it', binary=True)}"
             data-mlthd="{get_data_attr('mlthd_it', binary=True)}"
             data-mlsed="{get_data_attr('mlsed_it', binary=True)}"
             data-mlpubd="{get_data_attr('mlpubd_it', binary=True)}"
             data-mldnk="{get_data_attr('mldnk_it', binary=True)}"
             data-mlaccu="{get_data_attr('mlaccu_it')}"
             data-mlbias="{get_data_attr('mlbias_it')}"
             data-emrhrcrt="{get_data_attr('emrhrcrt_it', binary=True)}"
             data-sdhsn="{get_data_attr('sdhsn_it')}"
             data-snrec="{get_data_attr('snrec_it')}"
             data-sctool="{get_data_attr('sctool_it')}"
             >
        </div>
    """
    return icon_html


def add_interactive_controls_complete(m):
    """Complete interactive controls with all labels and legends"""
    
    control_html = """
    <style>
    .control-panel {
        position: fixed; top: 10px; right: 10px; z-index: 1000;
        background-color: white; padding: 15px; border-radius: 5px;
        box-shadow: 0 0 10px rgba(0,0,0,0.1); width: 320px; max-height: 90vh;
        overflow-y: auto;
    }
    
    .control-section {
        margin-bottom: 15px; padding-bottom: 15px; border-bottom: 1px solid #eee;
    }
    
    select { width: 100%; padding: 5px; margin-bottom: 5px; font-size: 13px; }
    h4 { margin: 0 0 10px 0; font-size: 14px; font-weight: bold; color: #2c3e50; }
    
    .submenu {
        display: none; margin-top: 10px;
    }
    
    .nested-submenu {
        display: none; margin-top: 10px; margin-left: 20px;
    }
    </style>
    
    <div class="control-panel">
        <div style="text-align: center; margin-bottom: 15px; font-weight: bold; font-size: 16px; color: #2c3e50;">
            AHA IT Survey 2023
        </div>
        
        <div class="control-section">
            <h4>Hospital Markers Visualization</h4>
            <select id="mainVizType" onchange="updateMainSelection()">
                <option value="">Choose visualization...</option>
                <option value="response">AHA IT Response Status</option>
                <option value="advanced_analytics">Advanced Analytics</option>
                <option value="ehr_systems_vendors">EHR Systems & Vendors</option>
                <option value="sdoh">Social Determinants of Health</option>
            </select>
            
            <div id="advancedAnalyticsSubMenu" class="submenu">
                <select id="analyticsType" onchange="updateAnalyticsSelection()">
                    <option value="">Choose category...</option>
                    <option value="clinicaldatausage">Clinical Data Usage</option>
                    <option value="ai">AI/ML Implementation</option>
                    <option value="mlusage">AI/ML Usage Types</option>
                    <option value="mldeveloper">AI/ML Developer</option>
                    <option value="mleval">AI/ML Evaluation</option>
                </select>
                
                <div id="clinicalDataUsageSubMenu" class="nested-submenu">
                    <select id="clinicalDataUsageType" onchange="updateVisualization()">
                        <option value="">Select type...</option>
                        <option value="equery">Clinician Query Data</option>
                        <option value="eadhg">Adherence to Guidelines</option>
                        <option value="eidgap">Identify Care Gap</option>
                        <option value="esimp">Quality Improvement</option>
                        <option value="emsaf">Monitor Patient Safety</option>
                        <option value="eralg">Identify High Risk</option>
                        <option value="enota">None</option>
                    </select>
                </div>
                
                <div id="mlUsageSubMenu" class="nested-submenu">
                    <select id="mlUsageType" onchange="updateVisualization()">
                        <option value="">Select type...</option>
                        <option value="aitraj">Predict Trajectory</option>
                        <option value="airfol">High Risk Patients</option>
                        <option value="aimhea">Monitor Health</option>
                        <option value="airect">Recommend Treatment</option>
                        <option value="aibill">Billing Automation</option>
                        <option value="aische">Scheduling</option>
                        <option value="aipoth">Operations</option>
                        <option value="aicloth">Other Clinical</option>
                        <option value="ainota">None</option>
                    </select>
                </div>
                
                <div id="mlDeveloperSubMenu" class="nested-submenu">
                    <select id="mlDeveloperType" onchange="updateVisualization()">
                        <option value="">Select type...</option>
                        <option value="mldev">EHR Developer</option>
                        <option value="mlthd">Third-party</option>
                        <option value="mlsed">Self-developed</option>
                        <option value="mlpubd">Public Domain</option>
                        <option value="mldnk">Don't Know</option>
                    </select>
                </div>
                
                <div id="mlEvalSubMenu" class="nested-submenu">
                    <select id="mlEvalType" onchange="updateVisualization()">
                        <option value="">Select metric...</option>
                        <option value="mlaccu">Accuracy</option>
                        <option value="mlbias">Bias</option>
                    </select>
                </div>
            </div>
            
            <div id="ehrSystemsVendorsSubMenu" class="submenu">
                <select id="EHRType" onchange="updateVisualization()">
                    <option value="">Choose category...</option>
                    <option value="emrhrcrt">Certified EHR Usage</option>
                    <option value="piemr">EHR Vendor</option>
                </select>
            </div>
            
            <div id="sdohSubMenu" class="submenu">
                <select id="sdohType" onchange="updateVisualization()">
                    <option value="">Choose category...</option>
                    <option value="sdhsn">SDOH Data Collection</option>
                    <option value="snrec">Recording Method</option>
                    <option value="sctool">Screening Tool Type</option>
                </select>
            </div>
        </div>
    </div>
    
    <div id="legend" style='position: fixed; bottom: 50px; right: 10px; z-index: 1000;
                           background-color: white; padding: 10px; border-radius: 5px;
                           box-shadow: 0 0 10px rgba(0,0,0,0.1); display: none;'>
    </div>
    
    <script>
    function updateMainSelection() {
        const mainType = document.getElementById('mainVizType').value;
        hideAllHospitalSubmenus();
        
        switch(mainType) {
            case 'advanced_analytics':
                document.getElementById('advancedAnalyticsSubMenu').style.display = 'block';
                break;
            case 'ehr_systems_vendors':
                document.getElementById('ehrSystemsVendorsSubMenu').style.display = 'block';
                break;
            case 'sdoh':
                document.getElementById('sdohSubMenu').style.display = 'block';
                break;
            case 'response':
                updateVisualization();
                break;
        }
    }
    
    function hideAllHospitalSubmenus() {
        document.getElementById('advancedAnalyticsSubMenu').style.display = 'none';
        document.getElementById('ehrSystemsVendorsSubMenu').style.display = 'none';
        document.getElementById('sdohSubMenu').style.display = 'none';
        hideNestedSubmenus();
    }
    
    function hideNestedSubmenus() {
        document.getElementById('clinicalDataUsageSubMenu').style.display = 'none';
        document.getElementById('mlUsageSubMenu').style.display = 'none';
        document.getElementById('mlDeveloperSubMenu').style.display = 'none';
        document.getElementById('mlEvalSubMenu').style.display = 'none';
    }
    
    function updateAnalyticsSelection() {
        const analyticsType = document.getElementById('analyticsType').value;
        hideNestedSubmenus();
        
        switch(analyticsType) {
            case 'clinicaldatausage':
                document.getElementById('clinicalDataUsageSubMenu').style.display = 'block';
                break;
            case 'mlusage':
                document.getElementById('mlUsageSubMenu').style.display = 'block';
                break;
            case 'ai':
                updateVisualization();
                break;
            case 'mldeveloper':
                document.getElementById('mlDeveloperSubMenu').style.display = 'block';
                break;
            case 'mleval':
                document.getElementById('mlEvalSubMenu').style.display = 'block';
                break;
        }
    }
    
    function updateVisualization() {
        const mainType = document.getElementById('mainVizType').value;
        const analyticsType = document.getElementById('analyticsType') ? document.getElementById('analyticsType').value : '';
        const EHRType = document.getElementById('EHRType') ? document.getElementById('EHRType').value : '';
        const sdohType = document.getElementById('sdohType') ? document.getElementById('sdohType').value : '';
        
        const clinicalDataUsageType = document.getElementById('clinicalDataUsageType') ? document.getElementById('clinicalDataUsageType').value : '';
        const mlUsageType = document.getElementById('mlUsageType') ? document.getElementById('mlUsageType').value : '';
        const mlDeveloperType = document.getElementById('mlDeveloperType') ? document.getElementById('mlDeveloperType').value : '';
        const mlEvalType = document.getElementById('mlEvalType') ? document.getElementById('mlEvalType').value : '';
        
        document.querySelectorAll('.hospital-marker').forEach(marker => {
            let backgroundColor = '#666666';
            
            if (mainType === 'response') {
                backgroundColor = getResponseColor(marker.getAttribute('data-response'));
            } 
            else if (mainType === 'advanced_analytics') {
                switch(analyticsType) {
                    case 'clinicaldatausage':
                        backgroundColor = getBinaryColor(marker.getAttribute('data-' + clinicalDataUsageType));
                        break;
                    case 'ai':
                        backgroundColor = getAIColor(marker.getAttribute('data-ai'));
                        break;
                    case 'mlusage':
                        backgroundColor = getBinaryColor(marker.getAttribute('data-' + mlUsageType));
                        break;
                    case 'mldeveloper':
                        backgroundColor = getBinaryColor(marker.getAttribute('data-' + mlDeveloperType));
                        break;
                    case 'mleval':
                        backgroundColor = getMLEvalColor(marker.getAttribute('data-' + mlEvalType));
                        break;
                }
            }
            else if (mainType === 'ehr_systems_vendors') {
                switch(EHRType) {
                    case 'emrhrcrt':
                        backgroundColor = getBinaryColor(marker.getAttribute('data-emrhrcrt'));
                        break;
                    case 'piemr':
                        backgroundColor = getPIEMRColor(marker.getAttribute('data-piemr'));
                        break;
                }
            }
            else if (mainType === 'sdoh') {
                switch(sdohType) {
                    case 'sdhsn':
                        backgroundColor = getsdhsnColor(marker.getAttribute('data-sdhsn'));
                        break;
                    case 'snrec':
                        backgroundColor = getsnrecColor(marker.getAttribute('data-snrec'));
                        break;
                    case 'sctool':
                        backgroundColor = getsctoolColor(marker.getAttribute('data-sctool'));
                        break;
                }
            }
            
            marker.style.backgroundColor = backgroundColor;
        });
        
        updateLegend(mainType, analyticsType, EHRType, sdohType);
    }
    
    function getResponseColor(value) {
        if (!value) return '#666666';
        return value === 'true' ? '#0066CC' : '#D73027';
    }
    
    function getAIColor(value) {
        if (!value) return '#666666';
        return value === '1.0' ? '#0066CC' :
               value === '2.0' ? '#009933' :
               value === '3.0' ? '#FF9900' :
               value === '4.0' ? '#FF9900' :
               '#666666';
    }
    
    function getBinaryColor(value) {
        if (!value) return '#666666';
        return value === '1' ? '#0066CC' : '#D73027';
    }
    
    function getMLEvalColor(value) {
        if (!value) return '#666666';
        return value === '1.0' ? '#0066CC' :
               value === '2.0' ? '#238B45' :
               value === '3.0' ? '#FEE08B' :
               value === '4.0' ? '#FDAE61' :
               value === '5.0' ? '#D73027' :
               value === '6.0' ? '#D73027' :
               '#666666';
    }
    
    function getPIEMRColor(value) {
    if (!value) return '#666666';
    value = value.trim();
    // Handle various formats and casing
    const valueLower = value.toLowerCase();
    
    if (valueLower.includes('cerner')) return '#00BFC4';
    if (valueLower.includes('epic')) return '#F8766D';
    if (valueLower.includes('meditech')) return '#7CAE00';
    if (valueLower.includes('alter')) return '#8B00FF';
    if (valueLower.includes('cpsi') || valueLower.includes('evident')) return '#8B4513';
    if (valueLower === 'other' || valueLower === 'others') return '#FFD700';
    
    // Fallback for any other vendor
    return '#FFD700';  // Other color
}
    
    function getsdhsnColor(value) {
        if (!value) return '#666666';
        return value === '1.0' ? '#0066CC' :
               value === '2.0' ? '#238B45' :
               value === '3.0' ? '#D73027' :
               value === '4.0' ? '#D73027' :
               '#666666';
    }
    
    function getsnrecColor(value) {
        if (!value) return '#666666';
        return value === '1.0' ? '#0066CC' :
               value === '2.0' ? '#238B45' :
               value === '3.0' ? '#FFD700' :
               value === '4.0' ? '#E66101' :
               '#666666';
    }
    
    function getsctoolColor(value) {
        if (!value) return '#666666';
        return value === '1.0' ? '#0066CC' :
               value === '2.0' ? '#238B45' :
               value === '3.0' ? '#D73027' :
               '#666666';
    }
    
    function updateLegend(mainType, analyticsType, EHRType, sdohType) {
        const legend = document.getElementById('legend');
        let legendContent = '';
        
        if (mainType === 'response') {
            legendContent = `<h4>AHA IT Response Status</h4>
                <div><span style='color: #0066CC'>●</span> Responded</div>
                <div><span style='color: #D73027'>●</span> No Response</div>
                <div><span style='color: #666666'>●</span> Missing</div>`;
        }
        else if (mainType === 'advanced_analytics') {
            switch(analyticsType) {
                case 'ai':
                    legendContent = `<h4>AI/ML Implementation</h4>
                        <div><span style='color: #0066CC'>●</span> Machine Learning</div>
                        <div><span style='color: #009933'>●</span> Non-ML Models</div>
                        <div><span style='color: #FF9900'>●</span> Neither/Unknown</div>
                        <div><span style='color: #666666'>●</span> Missing</div>`;
                    break;
                case 'clinicaldatausage':
                case 'mlusage':
                case 'mldeveloper':
                    legendContent = `<h4>${analyticsType.replace('clinicaldatausage', 'Clinical Data Usage').replace('mlusage', 'ML Usage').replace('mldeveloper', 'ML Developer')}</h4>
                        <div><span style='color: #0066CC'>●</span> Yes</div>
                        <div><span style='color: #D73027'>●</span> No</div>
                        <div><span style='color: #666666'>●</span> Missing</div>`;
                    break;
                case 'mleval':
                    legendContent = `<h4>ML Model Evaluation</h4>
                        <div><span style='color: #0066CC'>●</span> All Models</div>
                        <div><span style='color: #238B45'>●</span> Most Models</div>
                        <div><span style='color: #FEE08B'>●</span> Some Models</div>
                        <div><span style='color: #FDAE61'>●</span> Few Models</div>
                        <div><span style='color: #D73027'>●</span> None/Unknown</div>
                        <div><span style='color: #666666'>●</span> Missing</div>`;
                    break;
            }
        }
        else if (mainType === 'ehr_systems_vendors') {
            switch(EHRType) {
                case 'emrhrcrt':
                    legendContent = `<h4>Certified EHR Usage</h4>
                        <div><span style='color: #0066CC'>●</span> Yes</div>
                        <div><span style='color: #D73027'>●</span> No</div>
                        <div><span style='color: #666666'>●</span> Missing</div>`;
                    break;
                case 'piemr':
                    legendContent = `<h4>Primary EHR Vendor</h4>
                        <div><span style='color: #00BFC4'>●</span> Cerner</div>
                        <div><span style='color: #F8766D'>●</span> Epic</div>
                        <div><span style='color: #7CAE00'>●</span> Meditech</div>
                        <div><span style='color: #FFD700'>●</span> Other</div>
                        <div><span style='color: #8B00FF'>●</span> Altera</div>
                        <div><span style='color: #8B4513'>●</span> CPSI/Evident</div>
                        <div><span style='color: #666666'>●</span> Missing</div>`;
                    break;
            }
        }
        else if (mainType === 'sdoh') {
            switch(sdohType) {
                case 'sdhsn':
                    legendContent = `<h4>SDOH Data Collection</h4>
                        <div><span style='color: #0066CC'>●</span> Yes</div>
                        <div><span style='color: #238B45'>●</span> Not Routinely</div>
                        <div><span style='color: #D73027'>●</span> No/Unknown</div>
                        <div><span style='color: #666666'>●</span> Missing</div>`;
                    break;
                case 'snrec':
                    legendContent = `<h4>Recording Method</h4>
                        <div><span style='color: #0066CC'>●</span> Electronic Tool</div>
                        <div><span style='color: #238B45'>●</span> Free-text</div>
                        <div><span style='color: #FFD700'>●</span> Diagnosis Code</div>
                        <div><span style='color: #E66101'>●</span> Non-electronic</div>
                        <div><span style='color: #666666'>●</span> Missing</div>`;
                    break;
                case 'sctool':
                    legendContent = `<h4>Screening Tool</h4>
                        <div><span style='color: #0066CC'>●</span> Custom Tool</div>
                        <div><span style='color: #238B45'>●</span> External Tool</div>
                        <div><span style='color: #D73027'>●</span> Unknown</div>
                        <div><span style='color: #666666'>●</span> Missing</div>`;
                    break;
            }
        }
        
        legend.innerHTML = legendContent;
        legend.style.display = legendContent ? 'block' : 'none';
    }
    
    window.onload = function() {
        updateVisualization();
    }
    </script>
    """
    
    m.get_root().html.add_child(folium.Element(control_html))


# Now create maps for ALL hospitals (not just California)
print("\n" + "="*70)
print("Creating Maps for ALL Hospitals with Interactive Controls")
print("="*70)

# Configuration
MAP_CENTER = [39.8283, -98.5795]  # Center of USA
MAP_ZOOM = 4

# Prepare ALL hospital data (not filtered to CA)
all_data = AHA2023.copy()
all_data = all_data.dropna(subset=['lat_as', 'long_as'])
all_data['lat_as'] = pd.to_numeric(all_data['lat_as'], errors='coerce')
all_data['long_as'] = pd.to_numeric(all_data['long_as'], errors='coerce')
all_data = all_data.dropna(subset=['lat_as', 'long_as'])

# Ensure 'answered' column exists
if 'id_it' in all_data.columns:
    all_data['answered'] = all_data['id_it'].notnull()
else:
    all_data['answered'] = True

print(f"Using {len(all_data)} hospitals (all states)")

def add_hospitals_to_map(m, data):
    """Add hospital markers to map"""
    for idx, row in data.iterrows():
        lat = row['lat_as']
        lon = row['long_as']
        
        icon_html = create_hospital_marker_with_controls(row)
        
        popup_html = f"""
        <div style="font-family: Arial; min-width: 250px;">
            <h3 style="margin-top: 0; color: #2c3e50;">{row.get('mname_as', 'Unknown')}</h3>
            <p><b>City:</b> {row.get('mcity_as', 'Unknown')}</p>
            <p><b>State:</b> {row.get('mstate_as', 'Unknown')}</p>
            <p><b>County:</b> {row.get('mcounty_as', 'Unknown')}</p>
        </div>
        """
        
        folium.Marker(
            location=[lat, lon],
            icon=folium.DivIcon(html=icon_html, icon_size=(10, 10)),
            popup=folium.Popup(popup_html, max_width=300),
            tooltip=row.get('mname_as', 'Hospital')
        ).add_to(m)

hpsa_files = {}

for name, hpsa_data, filename in [
    ('HPSA Primary Care', hpsa_primary, 'map_primary.html'),
    ('HPSA Dental', hpsa_dental, 'map_dental.html'),
    ('HPSA Mental Health', hpsa_mental, 'map_mental.html'),
    ('None', None, 'map_none.html')
]:
    print(f"\nCreating {name} map...")
    
    # Create base map
    m = folium.Map(location=MAP_CENTER, zoom_start=MAP_ZOOM, tiles='CartoDB positron')
    
    # Add hospitals
    add_hospitals_to_map(m, all_data)
    
    # Add HPSA layer if not "None"
    if hpsa_data is not None:
        try:
            hpsa_crs = hpsa_data.to_crs("EPSG:4326")
            hpsa_crs['idx'] = range(len(hpsa_crs))
            hpsa_crs['score'] = pd.to_numeric(hpsa_crs['HpsScore'], errors='coerce')
            hpsa_clean = hpsa_crs.dropna(subset=['score'])
            
            if len(hpsa_clean) > 0:
                hpsa_geojson = hpsa_clean[['idx', 'score', 'geometry']].to_json()
                hpsa_data_simple = hpsa_clean[['idx', 'score']]
                
                folium.Choropleth(
                    geo_data=hpsa_geojson,
                    name=name,
                    data=hpsa_data_simple,
                    columns=['idx', 'score'],
                    key_on='feature.properties.idx',
                    fill_color='YlOrRd',
                    fill_opacity=0.7,
                    line_opacity=0.3,
                    legend_name=f'{name} Score',
                    show=True
                ).add_to(m)
                print(f"✓ Added {name} choropleth")
        except Exception as e:
            print(f"⚠ Error: {e}")
    
    # Add interactive controls
    add_interactive_controls_complete(m)
    
    # Save
    m.save(filename)
    hpsa_files[name] = filename
    print(f"✓ Saved: {filename}")

# Update container
container_html = """
<!DOCTYPE html>
<html>
<head>
    <title>US Hospitals - Interactive HPSA Map</title>
    <style>
        body {
            margin: 0;
            padding: 0;
            font-family: Arial, sans-serif;
            background-color: #f8f9fa;
        }
        .container {
            display: flex;
            flex-direction: column;
            height: 100vh;
        }
        .controls {
            background-color: white;
            padding: 20px;
            box-shadow: 0 2px 10px rgba(0,0,0,0.1);
            display: flex;
            flex-wrap: wrap;
            gap: 20px;
            align-items: center;
        }
        .control-group {
            display: flex;
            align-items: center;
            gap: 10px;
        }
        label {
            font-weight: bold;
            color: #2c3e50;
            font-size: 14px;
        }
        select {
            padding: 8px 15px;
            font-size: 14px;
            border: 2px solid #3498db;
            border-radius: 5px;
            background-color: white;
            color: #2c3e50;
            cursor: pointer;
            min-width: 250px;
        }
        select:hover {
            border-color: #2980b9;
        }
        .info-panel {
            margin-left: auto;
            background-color: #ecf0f1;
            padding: 10px 15px;
            border-radius: 5px;
            font-size: 13px;
            color: #7f8c8d;
        }
        .map-container {
            flex: 1;
            position: relative;
        }
        iframe {
            width: 100%;
            height: 100%;
            border: none;
        }
        #loading {
            position: absolute;
            top: 50%;
            left: 50%;
            transform: translate(-50%, -50%);
            background-color: rgba(255, 255, 255, 0.95);
            padding: 20px 40px;
            border-radius: 10px;
            box-shadow: 0 4px 20px rgba(0,0,0,0.3);
            display: none;
            font-size: 18px;
            color: #2c3e50;
        }
    </style>
</head>
<body>
    <div class="container">
        <div class="controls">
            <div class="control-group">
                <label for="layer-select">Background Choropleth:</label>
                <select id="layer-select" onchange="changeLayer()">
                    <option value="map_primary.html">HPSA Primary Care</option>
                    <option value="map_dental.html">HPSA Dental</option>
                    <option value="map_mental.html">HPSA Mental Health</option>
                    <option value="map_none.html">None</option>
                </select>
            </div>
            <div class="info-panel">
                <strong>Note:</strong> Use the control panel on each map (top-right) to visualize hospital IT survey responses
            </div>
        </div>
        <div class="map-container">
            <iframe id="map-frame" src="map_primary.html"></iframe>
            <div id="loading">Loading map...</div>
        </div>
    </div>
    
    <script>
        function changeLayer() {
            const select = document.getElementById('layer-select');
            const iframe = document.getElementById('map-frame');
            const loading = document.getElementById('loading');
            
            loading.style.display = 'block';
            iframe.src = select.value;
            
            iframe.onload = function() {
                loading.style.display = 'none';
            };
        }
        
        window.onload = function() {
            document.getElementById('map-frame').onload = function() {
                document.getElementById('loading').style.display = 'none';
            };
        };
    </script>
</body>
</html>
"""

with open('US_hospitals_container.html', 'w') as f:
    f.write(container_html)

print("\n✓ Updated container: US_hospitals_container.html")
print("\n" + "="*70)
print("Map files created with ALL hospitals (all states)")
print("="*70)
for name, filename in hpsa_files.items():
    print(f"✓ {filename} - {name}")
print("✓ US_hospitals_container.html - Container")

import webbrowser
import os
webbrowser.open('file://' + os.path.abspath('US_hospitals_container.html'))